# Get OpenAlex results into a pandas DataFrame
First we need to build the filter and get a response from the OpenAlex API

In [1]:
import pandas as pd
import requests
import plotly.express as px
from matplotlib import pyplot as plt

Set how many results you want per page in OpenAlex. Max is 200.

In [2]:
per_page = 4

Define a function that rebuilds the filter every time to ask for the next page when your total results are larger than a single page.

Adjust the filters for what you're looking for, then run this so we can call it later.

In [3]:
def build_filter(page):
    # build the 'filter' parameter
    filter_by_institution_id = 'institutions.ror:https://ror.org/03c4mmv16'
    filter_by_paratext = 'is_paratext:false'   # not cover, ToC, issue information, etc
    filter_by_type = 'type:dataset'
    filter_by_publication_date = 'from_publication_date:2012-07-20&per-page='+str(per_page)
    my_email = 'mailto=eschares@iastate.edu'
    page = 'page='+str(page)

    all_filters = (filter_by_institution_id, filter_by_paratext, filter_by_type, filter_by_publication_date)
    filter_param = f'filter={",".join(all_filters)}'
    filter_param = filter_param + '&' + my_email + '&' + page
    #print(f'filter query parameter:\n  {filter_param}')

    # put the URL together
    total_url = f'https://api.openalex.org/works?{filter_param}'
    #print(f'complete URL:\n  {total_url}')
    return total_url

To test it, pass a 1 to the `build_filter` function to make a query for the first page of results. 

The 1 you pass controls the "page=1" part at the end of the API URL

In [4]:
filtered_works_url = build_filter(1)
filtered_works_url

'https://api.openalex.org/works?filter=institutions.ror:https://ror.org/03c4mmv16,is_paratext:false,type:dataset,from_publication_date:2012-07-20&per-page=4&mailto=eschares@iastate.edu&page=1'

---

Send that test and get a response

In [5]:
api_response = requests.get(filtered_works_url)
parsed_response = api_response.json()

How many results?

In [6]:
parsed_response['meta']['count']

132

So how many pages will this take at the given per_page? 

(If it's less than a single page things start to get screwy, so try to set `per_page` so it goes beyond 1)

In [7]:
parsed_response['meta']['count'] / per_page

33.0

In [8]:
# If it takes 1.5 pages to give me all my records (for example), we'll need to ask it for 2
# To ask for 2, we'll need to set the range() to 3, since Python counts to stop-1

number_of_pages_needed = int(parsed_response['meta']['count'] / per_page) + 2
# need plus 2 to account for fractional page AND that python range stops 1 before end

# BUT if the total number of records is cleanly divisible by the per-page (say, 33.0 pages)
# we need to go back and remove one since the int() just drops the .0 and doesn't round
if (parsed_response['meta']['count'] % per_page) == 0:
    number_of_pages_needed -= 1
    
number_of_pages_needed

34

---
## Here is where I run the OpenAlex call over and over, getting multiple pages.

I save off the pieces of each record that I want to various lists using the .append; in my case I want the titles, years, DOIs, etc.

Then after I collect all that data, I combine them into a pandas dataframe

In [47]:
# clear the lists 
publication_titles = []
publication_year = []
publication_journal = []
publication_publisher = []
publication_doi = []

reference_years = []
reference_titles = []
reference_journal = []
reference_publisher = []
reference_doi = []

# go through all the pages of results
for page in range(1,number_of_pages_needed):   # offset already taken care of, don't need to +1 for range()
    print(f'Page {page}\n')                    # so if number_of_pages_needed is 3, variable 'page' will get 1 and 2
        
    # have to build the filter every time to ask for a new page
    filtered_works_url = build_filter(page)
    print(filtered_works_url)
    
    # Send it and get a response
    api_response = requests.get(filtered_works_url)
    parsed_response = api_response.json()

    # deal with fractional pages, need to keep track of how many leftover on the last page to avoid indexing error
    if page < (number_of_pages_needed-1):
        how_many_records = per_page
    else:  # figure out how many left on last page and only pass that many
        how_many_records = parsed_response['meta']['count'] - ((page-1) * per_page)
                                    # total records         -  pagesdone * records-per-page
        #print(f"page {page} num needed {number_of_pages_needed} how many records {how_many_records} total {parsed_response['meta']['count']} per page {per_page}")
        
    
    # Here is where you look into each record and pull out what you want
    
    for j in range(how_many_records):  # controls number of records to look at per page; 0 to per_page-1
        print(f"{parsed_response['results'][j]['title']}, {len(parsed_response['results'][j]['referenced_works'])} references")
        
        for i in parsed_response['results'][j]['referenced_works']:   #number of referenced works in the paper j
            splat = i.split('/')
            entity = splat[3]  #W1537479324

            single_work = requests.get('https://api.openalex.org/works/'+entity)
            parsed_single_work = single_work.json()

            # append the part you want to your lists
            reference_years.append(parsed_single_work['publication_year'])
            reference_titles.append(parsed_single_work['title'])
            reference_journal.append(parsed_single_work['host_venue']['display_name'])
            reference_publisher.append(parsed_single_work['host_venue']['publisher'])
            reference_doi.append(parsed_single_work['doi'])

            publication_year.append(parsed_response['results'][j]['publication_year'])
            publication_titles.append(parsed_response['results'][j]['title'])
            publication_journal.append(parsed_response['results'][j]['host_venue']['display_name'])
            publication_publisher.append(parsed_response['results'][j]['host_venue']['publisher'])
            publication_doi.append(parsed_response['results'][j]['doi'])

Page 1

https://api.openalex.org/works?filter=institutions.ror:https://ror.org/04rswrd78,is_paratext:false,type:journal-article,from_publication_date:2021-01-06,to_publication_date:2021-01-06&per-page=4&mailto=eschares@iastate.edu&page=1
X-Ray Flow Visualization in Multiphase Flows, 102 references
Synthetic glycosidases for the precise hydrolysis of oligosaccharides and polysaccharides, 45 references
Accurately Differentiating Between Patients With COVID-19, Patients With Other Viral Infections, and Healthy Individuals: Multimodal Late Fusion Learning Approach, 50 references
Effect of coil positioning and orientation of the quadruple butterfly coil during transcranial magnetic stimulation, 19 references
Page 2

https://api.openalex.org/works?filter=institutions.ror:https://ror.org/04rswrd78,is_paratext:false,type:journal-article,from_publication_date:2021-01-06,to_publication_date:2021-01-06&per-page=4&mailto=eschares@iastate.edu&page=2
How do neuroglial cells respond to ultrasound ind

---
Check of the list lengths, should be all the same for the dataframe to work

In [48]:
print(f'Length of referenced years: {len(reference_years)}')
print(f'Length of referenced titles: {len(reference_titles)}')
print(f'Length of referenced journals: {len(reference_journal)}')

print(f'Length of publication year: {len(publication_year)}')
print(f'Length of publication titles: {len(publication_titles)}')
print(f'Length of publication journals: {len(publication_journal)}')

Length of referenced years: 301
Length of referenced titles: 301
Length of referenced journals: 301
Length of publication year: 301
Length of publication titles: 301
Length of publication journals: 301


### Make the lists into a dataframe in pandas

In [49]:
# what you want to call the column : list name
d = {'publication':publication_titles,
     'publication_doi':publication_doi,
     'publication_year':publication_year,
     'publication_journal':publication_journal,
     'referenced_title':reference_titles,
     'reference_doi':reference_doi,
     'referenced_year':reference_years,
     'referenced_journal':reference_journal
    }
df = pd.DataFrame(data=d)  # make the dataframe
df.head(3)  #check it

,publication,publication_doi,publication_year,publication_journal,referenced_title,reference_doi,referenced_year,referenced_journal
0,X-Ray Flow Visualization in Multiphase Flows,https://doi.org/10.1146/annurev-fluid-010719-0...,2021,Annual Review of Fluid Mechanics,Industrial tomography using three different ga...,https://doi.org/10.1016/j.flowmeasinst.2015.10...,2016,Flow Measurement and Instrumentation
1,X-Ray Flow Visualization in Multiphase Flows,https://doi.org/10.1146/annurev-fluid-010719-0...,2021,Annual Review of Fluid Mechanics,Quantitative measurement of gas hold-up distri...,https://doi.org/10.1016/j.cej.2007.08.014,2008,Chemical Engineering Journal
2,X-Ray Flow Visualization in Multiphase Flows,https://doi.org/10.1146/annurev-fluid-010719-0...,2021,Annual Review of Fluid Mechanics,Gamma‐Ray Computed Tomography for Imaging of M...,https://doi.org/10.1002/cite.201200250,2013,Chemie Ingenieur Technik


In [50]:
# summary statistics of number columns
df.describe()

,publication_year,referenced_year
count,301.0,301.000000
mean,2021.0,2011.850498
std,0.0,9.790858
min,2021.0,1950.000000
25%,2021.0,2008.000000
50%,2021.0,2015.000000
75%,2021.0,2019.000000
max,2021.0,2021.000000
